In [1]:
import os, sys
script_dir = os.getcwd()
module_path = script_dir
for _ in range(1):
    module_path = os.path.abspath(os.path.join(module_path, '../'))
    if module_path not in sys.path:
        sys.path.insert(0,module_path)

from src import decode_moddeling, prefill_moddeling

import pandas as pd
from plotnine import *
import plotnine as p9
from src.system import System
import pandas as pd
from src.analye_model import *


/Users/abambhaniya3/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2   

In [3]:
model_df, summary_table = decode_moddeling(model = 'grok-1', batch_size = 1, Bb = 1 , bits='int8',
                            input_tokens = 1, output_tokens = 0, model_profilling=True,
                            tensor_parallel = 1, pipeline_parallel = 1, debug=True) 
display(model_df)
display(summary_table)
print(f'Model Weights={summary_table['Model Weights (MB)'][0]/1024/1024} T')
print(f'Attn Size:{(model_df.loc[0,'Input_w (MB)'] + model_df.loc[5,'Input_w (MB)'])*120/1024} B')
print(f'Each MLP Size:{sum(model_df.loc[6:,'Input_w (MB)'])*120/1024/16} B')
# print(f'FWD pass MLP Size:{sum(model_df.loc[6:,'Input_w (MB)'])*120/1024/16} B')

/Users/abambhaniya3/GenZ/src/LLM_inference/llm_decode.py:108: UserWarning: All params would not fit on chip. System Memory Cap:40.0 GB , Weights : 293.25 GB, KV Cache:0.0001220703125 


,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,GEMM,"[1, 8192, 1, 6144]",1.999431,100.663296,0.005859,48.0,0.007812,48.013672
1,Logit_MQA,"[1, 48, 1, 1, 128, 8]",1.702882,0.012288,0.005859,0.000977,0.000046,0.006882
2,Logit_MQA,"[1, 48, 1, 0, 128, 8]",0.0,0.0,0.005859,0.0,0.0,0.005859
3,Attend_MQA,"[1, 48, 1, 1, 128, 8]",1.702882,0.012288,0.000046,0.000977,0.005859,0.006882
4,Attend_MQA,"[1, 48, 1, 0, 128, 8]",0.0,0.0,0.0,0.0,0.005859,0.005859
5,GEMM,"[1, 6144, 1, 6144]",1.999349,75.497472,0.005859,36.0,0.005859,36.011719
6,GEMM,"[1, 98304, 1, 6144]",1.999654,1207.959552,0.005859,576.0,0.09375,576.099609
7,GEMM,"[1, 294912, 0, 6144]",0.0,0.0,0.0,1728.0,0.0,1728.0
8,GEMM,"[1, 6144, 1, 98304]",1.999654,1207.959552,0.09375,576.0,0.005859,576.099609
9,GEMM,"[1, 6144, 0, 294912]",0.0,0.0,0.0,1728.0,0.0,1728.0


,MACs (MFLOP),Total Data (MB),Model Weights (MB),Unused Weights (MB),KV Cache (MB),AR data (MB),Pipe data (MB)
0,165894.684672,300304.005859,300288.0,221184.0,0.125,0.75,0.0


Model Weights=0.286376953125 T
Attn Size:9.84375 B
Each MLP Size:33.75 B


In [4]:
for model in ['grok-1']:
    model_df, summary_table = prefill_moddeling(model = model, batch_size = 1,
                            input_tokens = 1024, output_tokens = 0, model_profilling=True,
                            tensor_parallel = 1, pipeline_parallel = 1) 
    print(model)
    display(model_df)
    display(summary_table)


grok-1


/Users/abambhaniya3/GenZ/src/LLM_inference/llm_prefill.py:105: UserWarning: All params would not fit on chip. System Memory Cap:40.0 GB , Weights : 586.5 GB, KV Cache:0.25 


,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,GEMM,"[1, 8192, 1024, 6144]",792.774194,103079.215104,12.0,96.0,16.0,124.0
1,Logit_MQA,"[1, 48, 1024, 1024, 128, 8]",111.709091,12884.901888,12.0,2.0,96.0,110.0
2,Attend_MQA,"[1, 48, 1024, 1024, 128, 8]",111.709091,12884.901888,96.0,2.0,12.0,110.0
3,GEMM,"[1, 6144, 1024, 6144]",768.0,77309.411328,12.0,72.0,12.0,96.0
4,GEMM,"[1, 393216, 256, 6144]",245.606496,1236950.581248,3.0,4608.0,192.0,4803.0
5,GEMM,"[1, 6144, 256, 393216]",245.606496,1236950.581248,192.0,4608.0,3.0,4803.0


,MACs (MFLOP),Total Data (MB),Model Weights (MB),KV Cache (MB),AR data (MB),Pipe data (MB)
0,1.715238e+08,642944.0,600576.0,256.0,1536.0,0.0
